In [1]:
import pandas as pd
import warnings
import itertools
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
#Load the dataset
df = pd.read_csv(r"/home/chin/Bureau/prog/Python/ogoxi/csv/Loire-Cher-Indre/Loire Forezienne/merged.csv", sep=";", parse_dates=['Date'], index_col="Date")
#DOnnée seulement de 2010 a 2014 pour cette balise
df = df.drop('K0700010',axis=1)

In [2]:

resampled_df = df.resample('15T').mean()
interpolate_df = resampled_df.interpolate(method="time")
interpolate_df = interpolate_df.dropna()
scaler = StandardScaler()
scalar_numpy = scaler.fit_transform(interpolate_df)
scalar_df = pd.DataFrame(data=scalar_numpy, index=interpolate_df.index.values, columns=interpolate_df.columns.values)

print("std:")
print(np.sqrt(np.var(interpolate_df)))
print(interpolate_df["2016-11-22 18:30:00"])

std:
K0567520    17.821550
K0614010    12.759688
K0744010    20.604609
K0643110     9.664660
K0663310     8.375212
K0673310    19.475257
H (cm)      14.558427
K0624510     1.601616
K0753210    24.247867
K0773220    26.807213
K0813020    18.016456
dtype: float64


KeyError: '2016-11-22 18:30:00'

In [36]:

from pandas import DataFrame
from pandas import concat

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, -1, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    cols.append(df.shift(-n_out))
    names += [('var%d(t+%d)' % (j+1, n_out)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg
time_lag_df = series_to_supervised(scalar_df, 8,2, True)
i=0
for columns in time_lag_df :
    
    print(i)
    print(columns)
    i = i+1

0
var1(t-8)
1
var2(t-8)
2
var3(t-8)
3
var4(t-8)
4
var5(t-8)
5
var6(t-8)
6
var7(t-8)
7
var8(t-8)
8
var9(t-8)
9
var10(t-8)
10
var11(t-8)
11
var1(t-7)
12
var2(t-7)
13
var3(t-7)
14
var4(t-7)
15
var5(t-7)
16
var6(t-7)
17
var7(t-7)
18
var8(t-7)
19
var9(t-7)
20
var10(t-7)
21
var11(t-7)
22
var1(t-6)
23
var2(t-6)
24
var3(t-6)
25
var4(t-6)
26
var5(t-6)
27
var6(t-6)
28
var7(t-6)
29
var8(t-6)
30
var9(t-6)
31
var10(t-6)
32
var11(t-6)
33
var1(t-5)
34
var2(t-5)
35
var3(t-5)
36
var4(t-5)
37
var5(t-5)
38
var6(t-5)
39
var7(t-5)
40
var8(t-5)
41
var9(t-5)
42
var10(t-5)
43
var11(t-5)
44
var1(t-4)
45
var2(t-4)
46
var3(t-4)
47
var4(t-4)
48
var5(t-4)
49
var6(t-4)
50
var7(t-4)
51
var8(t-4)
52
var9(t-4)
53
var10(t-4)
54
var11(t-4)
55
var1(t-3)
56
var2(t-3)
57
var3(t-3)
58
var4(t-3)
59
var5(t-3)
60
var6(t-3)
61
var7(t-3)
62
var8(t-3)
63
var9(t-3)
64
var10(t-3)
65
var11(t-3)
66
var1(t-2)
67
var2(t-2)
68
var3(t-2)
69
var4(t-2)
70
var5(t-2)
71
var6(t-2)
72
var7(t-2)
73
var8(t-2)
74
var9(t-2)
75
var10(t-2)
76
var11(

In [37]:
time_lag_df.columns[np.arange(100,110)]

Index([u'var2(t+2)', u'var3(t+2)', u'var4(t+2)', u'var5(t+2)', u'var6(t+2)',
       u'var7(t+2)', u'var8(t+2)', u'var9(t+2)', u'var10(t+2)', u'var11(t+2)'],
      dtype='object')

In [38]:
## import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

partial_time_lag = time_lag_df
#On enleve les colonnes > à l'instant T sauf la colonne objectif var2(t+n)
partial_time_lag = partial_time_lag.drop(partial_time_lag.columns[np.arange(100, 110)],axis=1)
#Permet d'obtenir un array d'index correspondant à 80% des enregistrements du dataset
msk = np.random.rand(len(partial_time_lag)) < 0.5
#Permet de séparer les features en deux datasets, un pour entrainer le modèle et l'autre pour le tester
train_X = partial_time_lag[msk].drop('var1(t+2)',axis=1)
test_X = partial_time_lag[~msk].drop('var1(t+2)',axis=1)

#Idem pour le label, càd l'objectif qu'on cherche à atteindre
train_Y = partial_time_lag['var1(t+2)'][msk]
test_Y =  partial_time_lag['var1(t+2)'][~msk]

# Use of sci-kit implemented regression methods----------------------
regr_lasso = linear_model.Lasso(0.05,max_iter=1500)
regr_ls = linear_model.LinearRegression()
# Calculations of the model coefficient-----------------------------
regr_lasso.fit(train_X,train_Y)
regr_ls.fit(train_X,train_Y)

# test of the model on unused data-----------------------------------
y_prediction_lasso = regr_lasso.predict(test_X)
y_prediction_ls = regr_ls.predict(test_X)


# creation of the result dataframes----------------------------
result_lasso = pd.DataFrame()
result_lasso['observed'] = test_Y
result_lasso['predicted'] = y_prediction_lasso

e_lasso = abs(result_lasso['observed'] - result_lasso['predicted']).max()

result_ls = pd.DataFrame()
result_ls['observed'] = test_Y
result_ls['predicted'] = y_prediction_ls

e_ls = abs(result_ls['observed'] - result_ls['predicted']).max()

# display of plots and interesting values-------------------

print("max difference lasso:" + str(e_lasso))
print("max difference ls:" + str(e_ls) +"\n")

print("Lasso regression coefficients:")
print(regr_lasso.coef_)
print("L1 norm of the coeff vector:" + str(sum(abs(regr_lasso.coef_))))
print("Variance of the prediction:" + str(np.var(result_lasso["predicted"])))
print("Mean squared error: %.2f"
      % mean_squared_error(test_Y, y_prediction_lasso))


print("Result of the lasso regression:")
print(result_lasso.head())


print("\n \nLeast square regression coefficients:")
print(regr_ls.coef_)
print("L1 norm of the coeff vector:" + str(sum(abs(regr_ls.coef_))))
print("Variance of the prediction:" + str(np.var(result_ls["predicted"])))
print("Mean squared error: %.2f"
      % mean_squared_error(test_Y, y_prediction_ls))
print("Result of the least square regression:")
print(result_ls.head())




max difference lasso:1.8112216431954284
max difference ls:1.7318692685012806

Lasso regression coefficients:
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.09637886 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.06255162 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.78837681 0.
 0.   

In [31]:
print(y)

NameError: name 'y' is not defined